# Lab | Data Aggregation and Filtering

In this challenge, we will continue to work with customer data from an insurance company. We will use the dataset called marketing_customer_analysis.csv, which can be found at the following link:

https://raw.githubusercontent.com/data-bootcamp-v4/data/main/marketing_customer_analysis.csv

This dataset contains information such as customer demographics, policy details, vehicle information, and the customer's response to the last marketing campaign. Our goal is to explore and analyze this data by first performing data cleaning, formatting, and structuring.

## Bonus

5. The marketing team wants to analyze the number of policies sold by state and month. Present the data in a table where the months are arranged as columns and the states are arranged as rows.

6.  Display a new DataFrame that contains the number of policies sold by month, by state, for the top 3 states with the highest number of policies sold.

*Hint:*
- *To accomplish this, you will first need to group the data by state and month, then count the number of policies sold for each group. Afterwards, you will need to sort the data by the count of policies sold in descending order.*
- *Next, you will select the top 3 states with the highest number of policies sold.*
- *Finally, you will create a new DataFrame that contains the number of policies sold by month for each of the top 3 states.*

7. The marketing team wants to analyze the effect of different marketing channels on the customer response rate.

Hint: You can use melt to unpivot the data and create a table that shows the customer response rate (those who responded "Yes") by marketing channel.

External Resources for Data Filtering: https://towardsdatascience.com/filtering-data-frames-in-pandas-b570b1f834b9

In [10]:
import pandas as pd

In [11]:
url = 'https://raw.githubusercontent.com/data-bootcamp-v4/data/main/marketing_customer_analysis.csv'
df = pd.read_csv(url)

In [12]:
df.head()

,Unnamed: 0,Customer,State,Customer Lifetime Value,Response,Coverage,Education,Effective To Date,EmploymentStatus,Gender,...,Number of Open Complaints,Number of Policies,Policy Type,Policy,Renew Offer Type,Sales Channel,Total Claim Amount,Vehicle Class,Vehicle Size,Vehicle Type
0,0,DK49336,Arizona,4809.216960,No,Basic,College,2/18/11,Employed,M,...,0.0,9,Corporate Auto,Corporate L3,Offer3,Agent,292.800000,Four-Door Car,Medsize,NaN
1,1,KX64629,California,2228.525238,No,Basic,College,1/18/11,Unemployed,F,...,0.0,1,Personal Auto,Personal L3,Offer4,Call Center,744.924331,Four-Door Car,Medsize,NaN
2,2,LZ68649,Washington,14947.917300,No,Basic,Bachelor,2/10/11,Employed,M,...,0.0,2,Personal Auto,Personal L3,Offer3,Call Center,480.000000,SUV,Medsize,A
3,3,XL78013,Oregon,22332.439460,Yes,Extended,College,1/11/11,Employed,M,...,0.0,2,Corporate Auto,Corporate L3,Offer2,Branch,484.013411,Four-Door Car,Medsize,A
4,4,QA50777,Oregon,9025.067525,No,Premium,Bachelor,1/17/11,Medical Leave,F,...,NaN,7,Personal Auto,Personal L2,Offer1,Branch,707.925645,Four-Door Car,Medsize,NaN


1. Create a new DataFrame that only includes customers who have a total_claim_amount greater than $1,000 and have a response of "Yes" to the last marketing campaign.

In [14]:
df_filtered = df.loc[(df['Total Claim Amount'] > 1000) & (df['Response'] == 'Yes')]
df_total_claim_yes = df_filtered[['Customer','Total Claim Amount','Response']].reset_index(drop=True)
df_total_claim_yes

,Customer,Total Claim Amount,Response
0,OK31456,1358.400000,Yes
1,YJ16163,1358.400000,Yes
2,GW43195,1027.200000,Yes
3,IP94270,1261.319869,Yes
4,FJ28407,1027.000029,Yes
...,...,...,...
62,FN44127,1176.278800,Yes
63,XZ64172,1324.800000,Yes
64,IX60941,1176.278800,Yes
65,QO62792,1008.000000,Yes


2. Using the original Dataframe, analyze the average total_claim_amount by each policy type and gender for customers who have responded "Yes" to the last marketing campaign. Write your conclusions.

In [16]:
df_customer_yes = df.loc[df['Response'] == 'Yes']
df_customer_yes['Response'].value_counts()
df_customer_yes.head(1)

,Unnamed: 0,Customer,State,Customer Lifetime Value,Response,Coverage,Education,Effective To Date,EmploymentStatus,Gender,...,Number of Open Complaints,Number of Policies,Policy Type,Policy,Renew Offer Type,Sales Channel,Total Claim Amount,Vehicle Class,Vehicle Size,Vehicle Type
3,3,XL78013,Oregon,22332.43946,Yes,Extended,College,1/11/11,Employed,M,...,0.0,2,Corporate Auto,Corporate L3,Offer2,Branch,484.013411,Four-Door Car,Medsize,A


In [34]:
df_grouped = df_customer_yes.groupby(['Policy Type','Gender'])['Total Claim Amount'].mean()
df_grouped

Policy Type     Gender
Corporate Auto  F         433.738499
                M         408.582459
Personal Auto   F         452.965929
                M         457.010178
Special Auto    F         453.280164
                M         429.527942
Name: Total Claim Amount, dtype: float64

In [ ]:
# Conclusion: En general, las mujeres pagan mas por sus polizas de seguro, independientemente del tipo de poliza.

3. Analyze the total number of customers who have policies in each state, and then filter the results to only include states where there are more than 500 customers.

In [72]:
df_state_counts = df.groupby('State')['Customer'].nunique()
states_more_than_500 = df_state_counts[df_state_counts > 500].reset_index()
states_more_than_500

,State,Customer
0,Arizona,1703
1,California,3150
2,Nevada,882
3,Oregon,2601
4,Washington,798


4. Find the maximum, minimum, and median customer lifetime value by education level and gender. Write your conclusions.

In [78]:
df_education_gender = df.groupby(['Education', 'Gender'])['Customer Lifetime Value'].agg(['max','min','median']).reset_index()
df_education_gender[['max', 'min', 'median']] = df_education_gender[['max', 'min', 'median']].round(2)
df_education_gender

,Education,Gender,max,min,median
0,Bachelor,F,73225.96,1904.00,5640.51
1,Bachelor,M,67907.27,1898.01,5548.03
2,College,F,61850.19,1898.68,5623.61
3,College,M,61134.68,1918.12,6005.85
4,Doctor,F,44856.11,2395.57,5332.46
5,Doctor,M,32677.34,2267.60,5577.67
6,High School or Below,F,55277.45,2144.92,6039.55
7,High School or Below,M,83325.38,1940.98,6286.73
8,Master,F,51016.07,2417.78,5729.86
9,Master,M,50568.26,2272.31,5579.10


In [ ]:
#Conclusion

#En general los que tienen una formacion menor pagan mas. Sin embargo en los valores minimos esta establecido una media poco dispersa
# Los valores de  mediana indican que realmente hay valores maximos dispersos ya que esta mediana se encuentra mas proxima a los minimos que a los maximos.